In [ ]:
import sys
sys.path.insert(0,"C:\\Program Files\\slab-master\\")
sys.path.insert(1,"C:\\Users\\MeasPC\\")

In [ ]:
from slab.instruments import PNAX
from slab import SlabFile

In [ ]:
pna=PNAX.N5242A(address="192.168.0.132")

In [ ]:
pna.get_id()

In [ ]:
def single_tone_CW(read_freq_center,span,read_power = read_power,probe_power=probe_power,ifbw = ifbw, sweep_pts= sweep_pts,avgs=avgs,
                   avgs_state=avgs_state,is_qubitdrive= 0,qfreq= 0,measurement="S21"):

    nwa.set_timeout(10e5)
    nwa.set_ifbw(ifbw)
    nwa.set_sweep_points(sweep_pts)
    nwa.clear_traces()
    nwa.setup_measurement(measurement)
    nwa.set_electrical_delay(delay, channel=1)

    vary_param = "single_tone_CW"
    print("Swept Parameter: %s" % (vary_param))
    prefix = "Readout_%s" % vary_param
    fname = get_next_filename(expt_path, prefix, suffix='.h5')
    print(fname)
    fname = os.path.join(expt_path, fname)

    nwa.setup_take(averages=avgs)
    nwa.set_average_state(avgs_state)

    read_freq_start = read_freq_center - span/2.0
    read_freq_stop =  read_freq_center + span/2.0

    print ("Setting up CW parameters")

    nwa.set_ifbw(ifbw)

    nwa.write('SENSE:FOM:STATE 1')
    nwa.write("sense:fom:range2:coupled 1")
    nwa.write("sense:fom:range3:coupled 1")
    nwa.write("sense:fom:range4:coupled 0")
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:START %f' % read_freq_start)
    nwa.write('SENSE:FOM:RANGE1:FREQUENCY:STOP %f' % read_freq_stop)

    nwa.write('SENSE:FOM:RANGE4:FREQUENCY:START %f' % qfreq)
    nwa.write('SENSE:FOM:RANGE4:FREQUENCY:STOP %f' % qfreq)
    
    nwa.write(":SOURCE:POWER1 %f" % (read_power))
    nwa.write(":SOURCE1:POWER1:MODE ON")
    print ("Read power",read_power)
    print ("Read Now On")


    #setting the port powers and decoupling the powers
    if measurement == 'S21':
        nwa.write(":SOURCE:POWER3 %f" % (probe_power))
    if is_qubitdrive:
        nwa.write(":SOURCE1:POWER3:MODE ON")
        print ("Qubit Drive Now ON")
    else:
        nwa.write(":SOURCE1:POWER3:MODE OFF")
        print ("Qubit Drive Now Off")

    data = nwa.take_one_in_mag_phase()
    fpoints = data[0]
    mags = data[1]
    phases = data[2]
    print ("finished downloading")
    
    with SlabFile(fname) as f:
        f.append_line('mags', mags)
        f.append_line('phases', phases)
        f.append_line('freq', fpoints)
        f.append_pt('read_power', read_power)
        f.append_pt('probe_power', probe_power)

    print (fname)
